<a href="https://colab.research.google.com/github/eloisacorti/ObjectDetection/blob/main/TBD_CovertAnnotations_YOLO_to_COCO_format.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This file converts object detection annotations from YOLO format (.txt and .names) to COCO format (.json)
# The base code is originally from: https://www.programmersought.com/article/76707275021/
# It was modified extensively as the original code was not working for my needs.
# More information about the conversion process can be found here: https://prabhjotkaurgosal.com/weekly-learning-blogs/

In [1]:
import os
import json
import cv2
import random
import time

In [2]:
#drive where we have our images and notations
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 Celular
'Colab Notebooks'
'curruculum (2).rar'
 CV
'CV. Ing. Industrial - Eloisa Cortiñas Bruno.pdf'
'CV-Ing. Industrial.Eloisa Elena Cortiñas.docx'
 Elo
'fotos aniversario'
'FOTOS CELULAR (07-11-2014)'
'FOTOS RIO'
'Fotos scaner'
 KIZOA-Movie-Maker-asmkm3a9.mp4
 MicroStrategy
 PANDAS
 Parecidos.wmv
 Peñaflor
 Peyte
'rebajahabilitacindecomercio (1).zip'
'Salvador de Bahía'
 TRAMONTO
 yolov3
 yolov4


In [3]:
#Create .names 
!echo -e 'maiz14' > ./obj.names


In [4]:
#Create a folder and unzip image dataset
!mkdir ./obj
!unzip /mydrive/yolov4/images.zip -d ./obj

Archive:  /mydrive/yolov4/images.zip
  inflating: ./obj/images/m13-m14-0000.jpg  
  inflating: ./obj/images/m13-m14-0000.txt  
  inflating: ./obj/images/m13-m14-0022.jpg  
  inflating: ./obj/images/m13-m14-0022.txt  
  inflating: ./obj/images/m13-m14-0025.jpg  
  inflating: ./obj/images/m13-m14-0025.txt  
  inflating: ./obj/images/m13-m14-0031.jpg  
  inflating: ./obj/images/m13-m14-0031.txt  
  inflating: ./obj/images/m13-m14-0034.jpg  
  inflating: ./obj/images/m13-m14-0034.txt  
  inflating: ./obj/images/m13-m14-0040.jpg  
  inflating: ./obj/images/m13-m14-0040.txt  
  inflating: ./obj/images/m13-m14-0043.jpg  
  inflating: ./obj/images/m13-m14-0043.txt  
  inflating: ./obj/images/m13-m14-0046.jpg  
  inflating: ./obj/images/m13-m14-0046.txt  
  inflating: ./obj/images/m13-m14-0049.jpg  
  inflating: ./obj/images/m13-m14-0049.txt  
  inflating: ./obj/images/m13-m14-0052.jpg  
  inflating: ./obj/images/m13-m14-0052.txt  
  inflating: ./obj/images/m13-m14-0055.jpg  
  inflating: ./obj

In [5]:
# The same for test
!mkdir ./val
!unzip /mydrive/yolov4/test_images.zip -d ./val

Archive:  /mydrive/yolov4/test_images.zip
  inflating: ./val/test_images/m13-m14-0028.jpg  
  inflating: ./val/test_images/m13-m14-0028.txt  
  inflating: ./val/test_images/m13-m14-0037.jpg  
  inflating: ./val/test_images/m13-m14-0037.txt  
  inflating: ./val/test_images/m13-m14-0064.jpg  
  inflating: ./val/test_images/m13-m14-0064.txt  
  inflating: ./val/test_images/m13-m14-0097.jpg  
  inflating: ./val/test_images/m13-m14-0097.txt  
  inflating: ./val/test_images/m13-m14-0196.jpg  
  inflating: ./val/test_images/m13-m14-0196.txt  
  inflating: ./val/test_images/m13-m14-0239.jpg  
  inflating: ./val/test_images/m13-m14-0239.txt  
  inflating: ./val/test_images/m13-m14-0396.jpg  
  inflating: ./val/test_images/m13-m14-0396.txt  
  inflating: ./val/test_images/m13-m14-1027.jpg  
  inflating: ./val/test_images/m13-m14-1027.txt  
  inflating: ./val/test_images/m13-m14-1221.jpg  
  inflating: ./val/test_images/m13-m14-1221.txt  
  inflating: ./val/test_images/m13-m14-1410.jpg  
  inflat

In [6]:
def yolo_to_coco(yolo_classes_path : str ,labels_directory :str ,images_directory:str) -> dict:
  '''
  Read the categories file and extract all categories
  Read the YOLO formatted label files (.txt) and images files (.jpg)
  to extarct bounding box information and store in COCO format
  '''
  directory_labels = os.fsencode(labels_directory)
  directory_images = os.fsencode(images_directory)

  with open(yolo_classes_path,'r') as f1:
    lines1 = f1.readlines()
  categories = []
  for j,label in enumerate(lines1):
      label = label.strip()
      categories.append({'id':j+1,'name':label,'supercategory': label})
      
  write_json_context = dict()
  write_json_context['info'] = {'description': '', 'url': '', 'version': '', 'year': 2022, 'contributor': '', 'date_created': '2022-12-28 10:00:08.5'}
  write_json_context['licenses'] = [{'id': 1, 'name': None, 'url': None}]
  write_json_context['categories'] = categories
  write_json_context['images'] = []
  write_json_context['annotations'] = []

  file_number = 1
  num_bboxes = 1
  for file in os.listdir(directory_images):
      filename = os.fsdecode(file)
      if filename.endswith(".jpg"):
          img_path = (os.path.join(directory_images.decode("utf-8"), filename))
          base=os.path.basename(img_path)
          file_name_without_ext = os.path.splitext(base)[0] # name of the file without the extension
          yolo_annotation_path  = os.path.join(directory_labels.decode("utf-8"), file_name_without_ext+ "." + 'txt')
          img_name = os.path.basename(img_path) # name of the file without the extension
          img_context = {}
          height,width = cv2.imread(img_path).shape[:2]
          img_context['file_name'] = img_name
          img_context['height'] = height
          img_context['width'] = width
          img_context['date_captured'] = '2022-12-28 11:00:08.5'
          img_context['id'] = file_number # image id
          img_context['license'] = 1
          img_context['coco_url'] =''
          img_context['flickr_url'] = ''
          write_json_context['images'].append(img_context)
          
          with open(yolo_annotation_path,'r') as f2:
              lines2 = f2.readlines() 

          for i,line in enumerate(lines2): # for loop runs for number of annotations labelled in an image
              line = line.split(' ')
              bbox_dict = {}
              class_id, x_yolo,y_yolo,width_yolo,height_yolo= line[0:]
              x_yolo,y_yolo,width_yolo,height_yolo,class_id= float(x_yolo),float(y_yolo),float(width_yolo),float(height_yolo),int(class_id)
              bbox_dict['id'] = num_bboxes
              bbox_dict['image_id'] = file_number
              bbox_dict['category_id'] = class_id+1
              bbox_dict['iscrowd'] = 0 # There is an explanation before
              h,w = abs(height_yolo*height),abs(width_yolo*width)
              bbox_dict['area']  = h * w
              x_coco = round(x_yolo*width -(w/2))
              y_coco = round(y_yolo*height -(h/2))
              if x_coco <0: #check if x_coco extends out of the image boundaries
                  x_coco = 1
              if y_coco <0: #check if y_coco extends out of the image boundaries
                  y_coco = 1
              bbox_dict['bbox'] = [x_coco,y_coco,w,h]
              bbox_dict['segmentation'] = [[x_coco,y_coco,x_coco+w,y_coco, x_coco+w, y_coco+h, x_coco, y_coco+h]]
              write_json_context['annotations'].append(bbox_dict)
              num_bboxes+=1
          
          file_number = file_number+1
          continue
      else:
          continue
  return write_json_context

In [7]:
yolo_classes = '/content/obj.names'
train_labels = '/content/obj/images'
train_images = '/content/obj/images'
test_lables =  '/content/val/test_images'
test_images =  '/content/val/test_images'

cocosave_path_train = './train.json'
cocosave_path_test = './test.json'

with open(cocosave_path_train,'w') as fw:
    json.dump(yolo_to_coco(yolo_classes,train_labels,train_images),fw)

with open(cocosave_path_test,'w') as fw:
    json.dump(yolo_to_coco(yolo_classes,test_lables,test_images),fw)